# 受講者の演習環境をバックアップします

受講者の演習環境をバックアップしたい場合に使用します。バックアップするファイル/フォルダを指定してバックアップします。

### 1. バックアップする受講者を指定します

バックアップ対象の受講者をメールアドレスで指定します。<br/>
バックアップファイルは、「メールアドレス_backup.tar.gz」のファイル名で、後述するバックアップの作業用フォルダに保存します。
```
STUDENTS=(
student1@example.com
student2@example.com
)
```

**次のセルで受講者のメールアドレスを設定して、セルを実行して下さい**

In [ ]:
STUDENTS=(
)

### 2. バックアップの作業用フォルダを作成します

* バックアップファイルを格納するフォルダを作成します。
* フォルダを変更する場合は、BACKUP_WORK_DIR を変更して下さい。
* フォルダが既存の場合、フォルダ配下のファイルは削除されます。必要があれば事前に退避して下さい。

**次のセルを実行して、作業用フォルダを作成して下さい。**

In [ ]:
BACKUP_WORK_DIR="$HOME/backup"
    
# create backup work directory
[ -e $BACKUP_WORK_DIR ] || {
    mkdir -p $BACKUP_WORK_DIR
    chown $USER:$USER $BACKUP_WORK_DIR
}
        
# creanlup backup work directory
rm -rf $BACKUP_WORK_DIR/*

### 3. バックアップするファイル/フォルダを指定します

* 受講者のホームディレクトリ配下でバックアップ対象とするファイルおよびフォルダを、BACKUP_LIST にスペース区切りで列挙して下さい。
```
BACKUP_LIST="*.ipynb work"
```
**次のセルでバックアップするファイル/フォルダを指定して、セルを実行して下さい。**

In [ ]:
BACKUP_LIST=""

### 4. バックアップに含めないファイル/フォルダを指定します

* バックアップに含めないファイルまたはフォルダがある場合は、EXCLUDES にスペース区切りで列挙して下さい。
```
EXCLUDES=".ipynb_checkpoints"
```
**次のセルで除外するファイル/フォルダを指定して、セルを実行して下さい。**

In [ ]:
EXCLUDES=""

### 5. 次のセルを実行して、バックアップファイルを作成して下さい

* バックアップファイルを生成します。
* バックアップファイルは、上記で作成したバックアップフォルダに、受講者毎のファイルで格納されます。

In [ ]:
# check setting
do_backup=1
if [ "${#STUDENTS[@]}" -eq 0 ]; then
    echo "STUDENTS is not set."
    do_backup=0
fi
if [ "$do_backup" -eq 1 ] && [ ! -e "$BACKUP_WORK_DIR" ]; then
    echo "BACKUP_WORK_DIR is not created."
    do_backup=0
fi
if [ "$do_backup" -eq 1 ] && [ -z "$BACKUP_LIST" ]; then
    echo "BACKUP_LIST is not set."
    do_backup=0
fi

#  do backup
if [ "$do_backup" -eq 1 ]; then
    exclude_optin=""
    if [ -n "$EXCLUDES" ]; then
        exlude_option="--exclude $EXCLUDES"
    fi
    for mail in ${STUDENTS[@]}; do
        userid=$(./ssh-auth-proxy << EOF
php -r "require_once '/var/www/php/functions.php'; echo get_username_from_mail_address('"$mail"');"
EOF
)
        user_dir="/jupyter/users/$userid"
        backup_file="$BACKUP_WORK_DIR/${userid}_backup.tar.gz"
        echo "$userid: $mail"
        (cd $user_dir; tar cvfz $backup_file $BACKUP_LIST $exclude_option)
    done
fi

echo
echo "... finished."